# HW 01 Data Wrangling
> BY: 新雅62/CDIE6 2016013327 项雨桐

> 2019-09-21

## 代码实现

In [399]:
# coding = <GBK>
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [400]:
with open("grades.html", encoding="GBK") as raw_html:
    parsed_html = BeautifulSoup(raw_html)
    
tables = parsed_html.find_all("table")

grades = tables[2]# no specific feature for the grade table, use numerical index to specify ( Will there be better choices using .find_all() if I can index a table by some particular strings?)

header, *records, _ = grades.tbody.find_all("tr", recursive = False) # the underscore is used for ignoring values, see https://hackernoon.com/understanding-the-underscore-of-python-309d1a029edc

fields = [field.div.text for field in header.find_all("th", recursive = False)]# maybe field.div just return an object and field.div.text is used for specify the string? cannot find clear explanation in official docs

In [401]:
def ParseCell(cell):
    if cell.div:
        if cell.div.script:# if there's script tag in div
            return cell.div.contents[0].strip() # what does contents mean?
        return cell.div.text.strip()
    else:
        return cell.text.strip()

data = [[ParseCell(cell) for cell in row.find_all("td")] for row in records]# form a list of lists for every row

data_frame = pd.DataFrame(columns=fields,data=data)

data_frame.to_csv("grades.csv",encoding = "GBK")

In [402]:
# so lucky (or not?) that I don't have any W/P/F's or any missing data in a row, so no other cleaning steps needed except for deleting empty columns. Still, I would put codes that handle the possible mess below, which have been validated by the given transcript)

# 1. drop the unnecessary columns

data_frame = data_frame.drop(columns = ['替代课程','特殊课程标志'])# note that by default it drops rows by labels

In [403]:
# 2. delete '成绩 = P' rows and '绩点 = N/A' 

data_frame = data_frame.drop(data_frame[(data_frame["成绩"] == "P")|(data_frame["成绩"] == "")|(data_frame["绩点"] == "N/A")].index)

In [404]:
# 3. string to numerical （学分，学时，绩点）for calculation
data_frame['学分'] = data_frame['学分'].replace("", np.nan)
data_frame['学分'] = pd.to_numeric(data_frame['学分'], downcast = 'integer')

data_frame['学时'] = data_frame['学时'].replace("", np.nan)
data_frame['学时'] = data_frame['学时'].fillna("0")#in case it turns out to be float
data_frame['学时'] = pd.to_numeric(data_frame['学时'], downcast = 'integer')

data_frame['绩点'] = data_frame['绩点'].replace("", np.nan)
data_frame['绩点'] = pd.to_numeric(data_frame['绩点'], downcast = 'float')

In [405]:
# 4. fill in missing '学时' by functional reliance
def replace_time(row):
    if row['学时'] == 0:
        row['学时'] = row['学分']*16
    return row

data_frame = data_frame.apply(replace_time, axis = 1)#apply send rows to func, or you can use lambda func

In [406]:
# calculate scores
def ap_to_score(letter):
    if letter == "A+":
        score = 100
    elif letter == "A":
        score = 97.5
    elif letter == "A":
        score = 97.5
    elif letter == "A-":
        score = 92
    elif letter == "B+":
        score = 87
    elif letter == "B":
        score = 82
    elif letter == "B-":
        score = 78
    elif letter == "C+":
        score = 74.5
    elif letter == "C":
        score = 91
    elif letter == "C-":
        score = 68
    elif letter == "D+":
        score = 64.5
    elif letter == "D":
        score = 61
    elif letter == "F":
        score = 0
    return score

In [407]:
# change arrangement of columns
data_frame['百分制成绩'] = data_frame.apply(lambda row: ap_to_score(row['成绩']), axis = 1)
new_fields = list(data_frame.columns.values)# get current column
new_fields.insert(7,'百分制成绩')
data_frame.reindex(new_fields,axis="columns")

,课程号,课序号,课程名,学分,学时,成绩,绩点,百分制成绩,课程属性,学年学期,考试时间,百分制成绩
0,12090043,6,军事理论与技能训练,3,144,B+,3.6,87.0,必修,2015-2016-3,20160820,87.0
1,10421055,5,微积分A(1),5,80,B+,3.6,87.0,必修,2016-2017-1,20170115,87.0
2,10421094,9,线性代数(1),4,78,B+,3.6,87.0,必修,2016-2017-1,20170115,87.0
3,10610183,17,思想道德修养与法律基础,3,48,B+,3.6,87.0,必修,2016-2017-1,20170115,87.0
4,10641082,97,学术英语读写（3）－文献综述,2,32,B-,3.0,78.0,必修,2016-2017-1,20170115,78.0
5,10720011,4,体育(1),1,32,C-,2.0,68.0,必修,2016-2017-1,20170115,68.0
6,10800163,90,艺术的启示,3,48,B+,3.6,87.0,限选,2016-2017-1,20170115,87.0
7,14700013,0,自我·他人·社会,3,64,B-,3.0,78.0,限选,2016-2017-1,20170115,78.0
8,14700041,0,大学之道,1,32,A-,4.0,92.0,限选,2016-2017-1,20161115,92.0
9,00640272,92,美国社会与文化,2,32,B+,3.6,87.0,必修,2016-2017-2,20170615,87.0


In [408]:
# get summary
summary_by_course_attribute_100 = data_frame.groupby(["课程属性"])[["百分制成绩"]].describe()
summary_by_course_attribute_4 = data_frame.groupby(["课程属性"])[["绩点"]].describe()

summary_by_semester_100 = data_frame.groupby(["学年学期"])[["百分制成绩"]].describe()
summary_by_semester_4 = data_frame.groupby(["学年学期"])[["绩点"]].describe()

# can be grouped by more than one category

# calculate average scores
average_score = sum(data_frame["百分制成绩"]*data_frame["学分"])/sum(data_frame["学分"])
average_ap = sum(data_frame["绩点"]*data_frame["学分"])/sum(data_frame["学分"])

# create score tube
tube = {'姓名':['项雨桐'], '班级':['新雅62/CDIE6'], '学号':['2016013327']}
tube['平均绩点'] = [round(average_ap,3)]
tube['平均成绩'] = [round(average_score,2)]
score_tube = pd.DataFrame.from_dict(tube)

## 结果显示

### 各类课程成绩

In [415]:
summary_by_course_attribute_100

百分制成绩                                                         
     count       mean       std   min     25%    50%     75%    max
课程属性                                                               
任选     2.0  94.750000  3.889087  92.0  93.375  94.75  96.125   97.5
必修    51.0  87.039216  8.213308  61.0  84.500  87.00  92.000  100.0
限选     7.0  89.357143  6.155717  78.0  87.000  92.00  92.000   97.5

In [416]:
summary_by_course_attribute_4

绩点                                             
     count      mean       std  min  25%  50%  75%  max
课程属性                                                   
任选     2.0  4.000000  0.000000  4.0  4.0  4.0  4.0  4.0
必修    51.0  3.509804  0.623299  1.3  3.3  3.6  4.0  4.0
限选     7.0  3.742857  0.377964  3.0  3.6  4.0  4.0  4.0

### 各学期成绩

In [412]:
summary_by_semester_100

百分制成绩                                                       
            count       mean        std   min    25%   50%    75%    max
学年学期                                                                    
2015-2016-3   1.0  87.000000        NaN  87.0  87.00  87.0  87.00   87.0
2016-2017-1   8.0  83.000000   7.745967  68.0  78.00  87.0  87.00   92.0
2016-2017-2  10.0  92.000000   3.519785  87.0  91.25  92.0  92.00   97.5
2016-2017-3   2.0  89.500000   3.535534  87.0  88.25  89.5  90.75   92.0
2017-2018-1   9.0  85.666667   6.619101  74.5  82.00  87.0  87.00   97.5
2017-2018-2  10.0  83.000000   8.640988  61.0  82.00  87.0  87.00   92.0
2017-2018-3   1.0  92.000000        NaN  92.0  92.00  92.0  92.00   92.0
2018-2019-1   9.0  89.611111   6.985600  74.5  87.00  92.0  92.00   97.5
2018-2019-2   9.0  89.666667  11.127107  64.5  87.00  92.0  97.50  100.0
2018-2019-3   1.0  97.500000        NaN  97.5  97.50  97.5  97.50   97.5

In [417]:
summary_by_semester_4

绩点                                             
            count      mean       std  min  25%  50%  75%  max
学年学期                                                          
2015-2016-3   1.0  3.600000       NaN  3.6  3.6  3.6  3.6  3.6
2016-2017-1   8.0  3.300000  0.623355  2.0  3.0  3.6  3.6  4.0
2016-2017-2  10.0  3.750000  0.535931  2.3  3.7  4.0  4.0  4.0
2016-2017-3   2.0  3.800000  0.282843  3.6  3.7  3.8  3.9  4.0
2017-2018-1   9.0  3.477778  0.426549  2.6  3.3  3.6  3.6  4.0
2017-2018-2  10.0  3.290000  0.747514  1.3  3.3  3.6  3.6  4.0
2017-2018-3   1.0  4.000000       NaN  4.0  4.0  4.0  4.0  4.0
2018-2019-1   9.0  3.711111  0.459468  2.6  3.6  4.0  4.0  4.0
2018-2019-2   9.0  3.611111  0.794425  1.6  3.6  4.0  4.0  4.0
2018-2019-3   1.0  4.000000       NaN  4.0  4.0  4.0  4.0  4.0

### 成绩条打印

In [418]:
score_tube

,姓名,班级,学号,平均绩点,平均成绩
0,项雨桐,新雅62/CDIE6,2016013327,3.671,88.75
